In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import itertools
import pyproj
from convertbng.util import convert_bng, convert_lonlat
import matplotlib.pyplot as plt
import ipdb

# InitialData

In [2]:
#raw data

post = pd.read_csv('Extra Data/open_postcode_elevation.csv', header=None)
orch = pd.read_csv('Extra Data/Traditional_Orchards_HAP.csv')

bees = pd.read_csv('Final Data/coords_species.csv') 
bees = bees.rename(columns = {'Lat':'lat', 'Long':'lon'})

shortbees = pd.read_csv('Final Data/shortbees.csv')
shortbees = shortbees.rename(columns = {'Lat':'lat', 'Long':'lon'})

C:\Users\moham\AppData\Local\Temp/ipykernel_7848/2863187422.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  post = pd.read_csv('Extra Data/open_postcode_elevation.csv', header=None)


In [3]:
shortbees

,lat,lon,Species,Present,Total
0,53.95,-1.0860,Honeybee,1,1887
1,50.96,-1.9200,Red-tailed Bumblebee,1,1442
2,51.04,-2.6670,Honeybee,1,1399
3,51.65,-1.8680,Common Carder,1,1294
4,51.49,0.2297,Honeybee,1,1281
...,...,...,...,...,...
12155,59.30,-2.9680,Honeybee,0,0
12156,59.30,-2.9680,Indet. Bee,0,0
12157,59.30,-2.9680,Red-tailed Bumblebee,0,0
12158,59.30,-2.9680,Tree Bumblebee,0,0


In [5]:
shortbees[:3, :]

InvalidIndexError: (slice(None, 3, None), slice(None, None, None))

In [51]:
coords = shortbees[['lat', 'lon']].drop_duplicates().reset_index()[['lat', 'lon']] #list of unique coordinates to create "dictionary"

# Functions

In [29]:
def coordPairs(df): # for all coordinates in the bees dataset, find all the nearest coordinates from the new datasets
    indexs = [] #keep track of nearest coordinate
    
    #"dictionary"
    for i, j in coords.iterrows(): 
        min_dist = 100000 #inital value
        c1 = (j['lat'], j['lon']) 
        
        for m, n in df[['lat', 'lon']].iterrows():
            c2 = (n['lat'], n['lon'])
            x = distance(c1, c2) #calculate distance
            
            if x == 0: #stop if theres an exact match
                index = m
                break
                
            if x <= min_dist: #find closest coords
                min_dist = x
                index = m
                
        indexs.append(index)
    
    # "dictionary"
    close = df.iloc[indexs].reset_index()
    pairs = pd.concat([coords, close[['index', 'lat', 'lon']]], axis = 1)

    return pairs

In [30]:
def fill(df, col, coord_pairs, op = None): #fill the bees dataset with the extra data using the nearest coordinates

    #filling data using dictionary
    final_indexs = []
    
    for i, j in shortbees.iterrows():
        lat = j['lat']
        lon = j['lon']
        
        q = coord_pairs.query('lat == @lat and lon == @lon')['index'].iloc[0]

        final_indexs.append(q)
        
    final_df = df.iloc[final_indexs].reset_index()
    
    if op == 'bin': #convert columns to binary indicators
        final_df[col] = np.where(final_df[col] >= 1, 1, 0)
        
    newdf = pd.concat([shortbees, final_df[col]], axis = 1)
    
    return newdf

In [31]:
def shrink(data, op = None): #round coordinates and group points with the same coordinates
    
    df = data.copy()
    
    df[['lat', 'lon']] = df[['lat', 'lon']].applymap(lambda x: round(x, 3 - int(np.floor(np.log10(abs(x))))))
    
    if op == 'av':
        grouped = df.groupby(['lat', 'lon']).mean().reset_index()
        
    else:
        grouped = df.groupby(['lat', 'lon']).sum().reset_index()
    
    return grouped

In [33]:
def distance(z1, z2): #calculate euclidian distance
    x = (z1[0] - z2[0])**2 + (z1[1] - z2[1])**2
    return x

# Elevation

In [12]:
#postcode
post = post[[1, 2]]
post.columns = ['postcode', 'elevation']

post.to_csv('postcodes.csv')

shortened = post[::5] #use every 5th row as observation becasue the dataset is too large

shortened.to_csv('short.csv', index=False)

In [52]:
#elevation
elev = pd.read_csv('elev.csv') #shortened was inputted into a program that took postcodes and returned their coordinates and the output is elev

elev = elev.drop(columns=['postcode'])

elev = elev.rename(columns = {'Latitude':'lat', 'Longitude':'lon'})

In [53]:
elev_shrink = shrink(elev, 'av')

In [162]:
elev_pairs = coordPairs(elev_shrink)

In [166]:
elev_pairs.to_csv('elev_pairs.csv')

In [54]:
elev_pairs = pd.read_csv('elev_pairs.csv')

elev_pairs = elev_pairs.drop(columns = ['Unnamed: 0'])

In [55]:
shortbees = fill(elev_shrink, ['elevation'], elev_pairs.iloc[:, 0:3]) #adding elevation data too the bee dataset

# Orchard

In [11]:
#orch
#cleaning inconsistent values in the dataset

orch = orch.drop(columns=['OBJECTID', 'Prihabtxt', 'Habdefver', 'S5OSMM', 'NonTOcode', 'OS_Epoch2', 'Area_Ha', 
                          'Shape_Length', 'Shape__Area', 'Shape__Length', 'S1GroundSv', 'S1Status', 'S1Planting',
                          'S2Status', 'S2Planting', 'S3Status', 'S3Planting', 'S4Status', 'S5OSMM', 'Stewardshp', 
                          'S2OrchQues'])

orch['lon'], orch['lat'] = convert_lonlat(orch['Easting'], orch['Northing'])

orch['S3Habid'] = orch['S3Habid'].replace(['Primary', 'Secondary', 'Tertiary', ' ', 'Other'], 
                                          [1, 2, 3, np.nan, np.nan])

orch['S4Habid'] = orch['S4Habid'].replace(['Primary', 'Secondary', 'Tertiary', 'Other', ' '], 
                                          [1, 2, 3, np.nan, np.nan])

orch['InterQual'] = orch['InterQual'].replace(['3 (Medium)', 'Medium (3)', '3'], 3)
orch['InterQual'] = orch['InterQual'].replace(['1 (High)', 'High (1)', '1', '1 (high)'], 1)
orch['InterQual'] = orch['InterQual'].replace(['2 (Medium)', '2'], 2)
orch['InterQual'] = orch['InterQual'].replace(['4 (Medium)'], 4)
orch['InterQual'] = orch['InterQual'].replace([' '], np.nan)

orch['PriDet'] = orch['PriDet'].replace(['Definitely is Traditional Orchard priority habitat', 
                                        'Priority Traditional Orchard habitat may be present but evidence is either insufficient to determine presence confidently or is in the oldest allowable category',
                                        'Probably Traditional Orchard priority habitat but some uncertainty due to age of data source'], [3,1,2])

orch['HabCon'] = orch['HabCon'].replace(['Excellent', 'Good', 'Poor', ' '], [3, 2, 1, np.nan])

orch['S1Habid'] = orch['S1Habid'].replace(['Primary', 'Secondary', 'SecondaryS', ' ', '25/09/2008'], 
                                          [1, 2, 2, np.nan, np.nan])

orch['S2Habid'] = orch['S2Habid'].replace(['Primary', 'Secondary', '-', ' ', '26/02/2010', '06/09/2012'], 
                                          [1, 2, np.nan, np.nan, np.nan, np.nan])

orch['S2Habid'] = orch['S2Habid'].replace(['Primary', 'Secondary', '-', ' ', '26/02/2010', '06/09/2012'], 
                                          [1, 2, np.nan, np.nan, np.nan, np.nan])

orch['S3Boundary'] = orch['S3Boundary'].replace(['Primary', 'Secondary', ' ', '01/06/2009', '12/05/2006'], 
                                          [1, 2, np.nan, np.nan, np.nan])

orch['S4Boundary'] = orch['S4Boundary'].replace(['Primary', 'Secondary', 'Tertiary', 'SecondaryS', 'Tertiart'], 
                                          [1, 2, 3, 2, 3])

orch['S5Boundary'] = orch['S5Boundary'].replace(['Primary', 'Secondary', 'Tertiary', '18/09/2004', ' ', 'Boundary u'], 
                                          [1, 2, 3, np.nan, np.nan, np.nan])

for i in ['Apple', 'Pear', 'Plum', 'Cherry', 'Damson', 'VetTrees10', 
          'Vet_11_30', 'Vet_31_100', 'Vet_100', 'YFT10', 'YFT_11_30', 'YFT_31_100', 
          'YFT_100', 'Grazed', 'Mown', 
          'Unmanaged', 'Sheep', 'Cattle', 'Equine', 'Pigs', 'Fowl', 'GrazingDmg', 
          'BranchHols', 'Loose_bark', 'Cavities', 'DW_Canopy', 'DW_Floor', 'WaterPools', 
          'Crevices', 'Sap_runs', 'FungalFrts', 'AerialRts', 'Hedgerows', 'Ponds', 'VetTrees', 'RoughAreas']:
    orch[i] = orch[i].replace(['1', 'y', 'Y'], 1)
    orch[i] = orch[i].replace(['?', '3', '6', '2', 'S', 's', 'D'], np.nan)
    orch[i] = orch[i].replace([' ', 'N', '0'], 0)

#features kept and also dropped any rows that have any missing values
orch = orch[['Apple', 'Pear', 'Plum', 'Cherry', 'Damson', 'VetTrees10', 
             'Vet_11_30', 'Vet_31_100', 'Vet_100', 'YFT10', 'YFT_11_30', 
             'YFT_31_100', 'YFT_100', 'Grazed', 'Mown', 'Unmanaged', 
             'Sheep', 'Cattle', 'Equine', 'Pigs', 'Fowl', 'GrazingDmg', 
             'BranchHols', 'Loose_bark', 'Cavities', 'DW_Canopy', 'DW_Floor', 
             'WaterPools', 'Crevices', 'Sap_runs', 'FungalFrts', 'AerialRts', 
             'Hedgerows', 'Ponds', 'VetTrees', 'RoughAreas', 'lat', 'lon']].dropna()

In [39]:
orch_shrink = shrink(orch, 'av')

In [42]:
orch_pairs = coordPairs(orch_shrink)

In [43]:
orch_pairs.to_csv('orch_pairs_av.csv')

In [47]:
orch_pairs = pd.read_csv('orch_pairs.csv')

orch_pairs = orch_pairs.drop(columns = ['Unnamed: 0'])

In [57]:
shortbees = fill(orch_shrink, orch_shrink.columns[2:], orch_pairs.iloc[:, 0:3]) #adding the new data to the bee dataset

In [58]:
shortbees.to_csv('final_bees_av.csv')